# Libraries and Data Loading

In [1]:
!pip install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357287 sha256=259a28ab460688cd2f7a2a499fb02d6ecb57e3a284448828c3a1a8c71629aaf5
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from surprise import Dataset, Reader, SVD, KNNBasic
from surprise.model_selection import cross_validate
from surprise import accuracy
from surprise.model_selection import train_test_split as surprise_train_test_split

# Load datasets
anime = pd.read_csv("anime.csv")  # replace with your file path
rating = pd.read_csv("rating.csv")  # replace with your file path

# Filter out rows with invalid ratings (-1 is often used to indicate missing ratings)
rating = rating[rating['rating'] > 0]
rating = rating.sample(n=20000, random_state=42).reset_index(drop=True)

# Prepare Data for Collaborative Filtering

In [3]:
reader = Reader(rating_scale=(1, 10))  # Assuming ratings are between 1 and 10
data = Dataset.load_from_df(rating[['user_id', 'anime_id', 'rating']], reader)

# User-Based Collaborative Filtering

In [4]:
# Split into train and test sets
trainset, testset = surprise_train_test_split(data, test_size=0.2, random_state=42)

# Define user-based collaborative filtering
user_cf = KNNBasic(sim_options={'user_based': True})
user_cf.fit(trainset)

# Predict on the test set
predictions_user = user_cf.test(testset)

# Calculate evaluation metrics
mse_user = accuracy.mse(predictions_user)
mae_user = accuracy.mae(predictions_user)
print(f"User-based CF -> MSE: {mse_user}, MAE: {mae_user}")


Computing the msd similarity matrix...
Done computing similarity matrix.
MSE: 2.3927
MAE:  1.2240
User-based CF -> MSE: 2.392676343267578, MAE: 1.22399928125


# Item-Based Collaborative Filtering

In [5]:
# Define item-based collaborative filtering
item_cf = KNNBasic(sim_options={'user_based': False})
item_cf.fit(trainset)

# Predict on the test set
predictions_item = item_cf.test(testset)

# Calculate evaluation metrics
mse_item = accuracy.mse(predictions_item)
mae_item = accuracy.mae(predictions_item)
print(f"Item-based CF -> MSE: {mse_item}, MAE: {mae_item}")


Computing the msd similarity matrix...
Done computing similarity matrix.
MSE: 2.3814
MAE:  1.2212
Item-based CF -> MSE: 2.381426343267578, MAE: 1.22124928125


In [7]:
import pickle
import pandas as pd

# Save the trained collaborative filtering model (item_cf)
with open('item_cf_model.pkl', 'wb') as f:
    pickle.dump(item_cf, f)

# Save the anime dataset
anime.to_pickle('anime_data.pkl')

# Recommendation System

In [9]:
import pandas as pd
import pickle

# Load the collaborative filtering model and anime data
with open('item_cf_model.pkl', 'rb') as f:
    item_cf = pickle.load(f)

anime = pd.read_pickle('anime_data.pkl')

In [10]:
def get_top_recommendations(anime_id=None, anime_name=None, top_n=10):
    # Map anime name to anime_id if anime_name is provided
    if anime_name:
        anime_id = anime[anime['name'].str.lower() == anime_name.lower()]['anime_id'].values
        if len(anime_id) == 0:
            return f"Anime '{anime_name}' not found in the dataset."
        anime_id = anime_id[0]

    if anime_id is None:
        return "Please provide a valid anime ID or name."

    # Get inner ID for the given anime
    inner_id = item_cf.trainset.to_inner_iid(anime_id)

    # Find similar items using collaborative filtering
    neighbors = item_cf.get_neighbors(inner_id, k=top_n)

    # Convert inner IDs back to raw IDs
    recommended_ids = [item_cf.trainset.to_raw_iid(neighbor) for neighbor in neighbors]

    # Retrieve details for recommended anime
    recommended_anime = anime[anime['anime_id'].isin(recommended_ids)].copy()

    # Return only top N recommendations
    return recommended_anime[['anime_id', 'name', 'genre', 'type', 'rating', 'members']].head(top_n)


In [11]:
recommendations = get_top_recommendations(anime_id=5114, top_n=10)
recommendations

,anime_id,name,genre,type,rating,members
112,136,Hunter x Hunter,"Action, Adventure, Shounen, Super Power",TV,8.48,166255
124,5341,Ookami to Koushinryou II,"Adventure, Fantasy, Historical, Romance",TV,8.46,210491
403,21855,Hanamonogatari,"Comedy, Mystery, Supernatural",TV,8.11,124886
468,268,Golden Boy,"Adventure, Comedy, Ecchi",OVA,8.05,113040
501,4163,Seto no Hanayome OVA,"Comedy, Parody, Romance, School",OVA,8.03,31973
687,552,Digimon Adventure,"Action, Adventure, Comedy, Fantasy, Kids",TV,7.89,182208
793,6347,Baka to Test to Shoukanjuu,"Comedy, Romance, School, Super Power",TV,7.83,301282
815,21659,Kill la Kill Special,"Action, Comedy, School, Super Power",Special,7.82,83795
3123,11285,Black★Rock Shooter (TV),"Action, Drama, School, Slice of Life",TV,7.07,198991
3185,31374,Shingeki! Kyojin Chuugakkou,"Comedy, Parody, School, Shounen",TV,7.06,52928


In [12]:
recommendations = get_top_recommendations(anime_name="Fullmetal Alchemist: Brotherhood", top_n=10)
recommendations

,anime_id,name,genre,type,rating,members
112,136,Hunter x Hunter,"Action, Adventure, Shounen, Super Power",TV,8.48,166255
124,5341,Ookami to Koushinryou II,"Adventure, Fantasy, Historical, Romance",TV,8.46,210491
403,21855,Hanamonogatari,"Comedy, Mystery, Supernatural",TV,8.11,124886
468,268,Golden Boy,"Adventure, Comedy, Ecchi",OVA,8.05,113040
501,4163,Seto no Hanayome OVA,"Comedy, Parody, Romance, School",OVA,8.03,31973
687,552,Digimon Adventure,"Action, Adventure, Comedy, Fantasy, Kids",TV,7.89,182208
793,6347,Baka to Test to Shoukanjuu,"Comedy, Romance, School, Super Power",TV,7.83,301282
815,21659,Kill la Kill Special,"Action, Comedy, School, Super Power",Special,7.82,83795
3123,11285,Black★Rock Shooter (TV),"Action, Drama, School, Slice of Life",TV,7.07,198991
3185,31374,Shingeki! Kyojin Chuugakkou,"Comedy, Parody, School, Shounen",TV,7.06,52928
